# 1. Convert JSON to DataFrame

In [27]:
import pandas as pd
import json
import re
import os
import math

def problem_unification(problem_str):
    ### this function unifies the column 'problem' between GPT-3 and Google's LMs datasets
    # e.g., problem_str = 'mult_data_wrangling:some_before_at_na_emails_1'
    problem = re.findall(r":(.+)_[a-z]+_[\d]+", problem_str)[0]
    problem_parts = problem.split('_')
    problem_str = problem_parts[0]
    
    for i, part in enumerate(problem_parts[1:]):
        i += 1
        if i == len(problem_parts) - 1:
            if part == 'na':
                part = 'NA'
            problem_str = problem_str + '-' + part
        else:
            problem_str = problem_str + part[0].upper() + part[1:] 
            
    return problem_str


def convert_JSONs_to_df(directory):
    path_to_json = directory
    json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
    # print(json_files)
    lis = []
    lis_prob = []
    for json_file in json_files:
        # print(json_file)
        with open(directory + json_file, encoding="utf8") as json_obj:
            data = json.load(json_obj)
            engine = data['model']['model_family'] + ' ' + data['model']['model_name']
            # non_embedding_params = data['model']['non_embedding_params']
            total_params = data['model']['total_params']
            
            
            for query in data['queries']:
                if query['function'] == 'generate_text': 
                    for sample in query['samples']:
                        problem = problem_unification(query['task'] )
                        domain = re.findall(r"_([a-z]+)_[\d]+$", query['task'])[0]
                        nshot = query['shots']
                        q = sample['input']   # query's input 
                        target = [t for t in sample['targets'].keys()][0]
                        output = sample['output']

 
                        correct = 1 if int([t for t in sample['targets'].values()][0]['exact_str_match']) == 1 else 0

                        lis.append([problem, domain, engine, nshot, q, target, 
                                    output, correct, total_params])
                else:
                    for sample in query["samples"]:
                        lis_prob.append(math.exp(sample["absolute_scores"][0]))
    df = pd.DataFrame(lis)
    df.columns = ['problem', 'domain', 'engine', 'nshot', 'query', 
                   'target', 'output', 'correct', 'total_params']
    df['prob'] = lis_prob
    return df
    
df = convert_JSONs_to_df('./datasets/full-logs_BIG-G_BIG-G-sparse_JSON_230422/')

In [30]:
"""
I have to:
1. extract the domain --> Done!
2. unify the column 'problem' for both datasets  --> Done!
3. run all the files for different LMs of Google  --> Done!
4. extract non_embedding_params and total_params for GPT3 dataset (of Nando) --> Done!
5. Then, try to think about what to do with the 'query' column


Notes:

Google's LMs only have up to 3shots 
Can different shots have different meta-features?
Then, maybe we should provide more meta-features (a set for each shot)
"""


df

,problem,domain,engine,nshot,query,target,output,correct,total_params,prob
0,addPunctuation-1,dates,BIG-G sparse 125m,0,Input: '110892' Output:,'11-08-92','110892',0,1777677312,4.325552e-09
1,addPunctuation-1,dates,BIG-G sparse 125m,0,Input: '021097' Output:,'02-10-97','021097',0,1777677312,9.040107e-05
2,addPunctuation-1,dates,BIG-G sparse 125m,0,Input: '270906' Output:,'27-09-06','270906',0,1777677312,8.287534e-10
3,addPunctuation-1,dates,BIG-G sparse 125m,0,Input: '030796' Output:,'03-07-96','030796',0,1777677312,1.314212e-04
4,addPunctuation-1,dates,BIG-G sparse 125m,1,Input: '290386' Output: '29-03-86'\n\nInput: '...,'11-08-92','110892',0,1777677312,1.050140e-04
...,...,...,...,...,...,...,...,...,...,...
133819,toUpper-2,freetext,BIG-G T=1 8b,2,the input string/s 'Aquila non capit muscas.' ...,'NEMO MORTALIUM OMNIBUS HORIS SAPIT','NEMO MORTIS OMNIBUS HORIS SAPIT.',0,8721174528,1.032499e-01
133820,toUpper-2,freetext,BIG-G T=1 8b,3,the input string/s 'Lorem ipsum dolor sit amet...,'TIBI BENE VOLO','TIBI BENE VOLO',1,8721174528,5.012325e-01
133821,toUpper-2,freetext,BIG-G T=1 8b,3,the input string/s 'Integer malesuada elit et ...,'FERE LIBENTER HOMINES ID QUOD VOLUNT CREDUNT','FERE LIBENTER HOMINIS ID QUOD VOLUNT CREDIT',0,8721174528,1.070722e-03
133822,toUpper-2,freetext,BIG-G T=1 8b,3,the input string/s 'Quid infantes sumus.' is/a...,'PLACE DICES MIHI LATINE','Place dices mihi LATIN',0,8721174528,5.949777e-02


In [31]:
df.to_csv('./datasets/Google.DataWrangling.csv')  

# 2. Convert query columns into "input_{i}" and "output_{i}" columns

## 2.1. Google's LMs

In [32]:
import re
import numpy as np

N = len(df['query'])
Inputs = [[np.NaN] * N for i in range(11)]
Outputs = [[np.NaN] * N for i in range(10)]
PromptFormat = [np.NaN] * N

ok = []
for i in range(N):
    row = df['query'][i]
    
    if row[:6] == 'Input:':
        PromptFormat[i] = 2
        res = [val for val in re.split(r'\n\n', df['query'][i])]
        for j, lis in enumerate(res):
            
            if j == df['nshot'][i]:   # The test input
                Inputs[-1][i] = re.findall(r"Input: '(.+)' Output: ", lis)[0].replace("', '", ", ")
            
            else:  # one of the nshot input
                Inputs[j][i] = re.findall(r"Input: '(.+)' Output", lis)[0].replace("', '", ", ")
                
                if len(re.findall(r"Output: '(.+)'", lis)) != 0:
                    Outputs[j][i] = re.findall(r"Output: '(.+)'", lis)[0].replace("', '", ", ")
                    
    else:     # row[:3] == 'the':
        PromptFormat[i] = 3
        res = re.split(';', row)
        for j, lis in enumerate(res):
            
            if j == df['nshot'][i]:
                Inputs[-1][i] = re.findall(r"the input string/s '(.+)' is/are formatted as ", lis)[0].replace("', '", ", ")
            
            else:
                Inputs[j][i] = re.findall(r"the input string/s '(.+)' is/are formatted as ", lis)[0].replace("', '", ", ")
                
                if len(re.findall(r"is/are formatted as '(.+)'", lis)) != 0:
                    Outputs[j][i] = re.findall(r"is/are formatted as '(.+)'", lis)[0].replace("', '", ", ")

print(Inputs)
print(Outputs)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [33]:
PromptFormat_df = pd.DataFrame(PromptFormat)
PromptFormat_df.columns = ["prompt_format"]

input_df = pd.DataFrame(Inputs).transpose()
input_df.columns = ["input"+str(i) for i in range(1,12)]
input_df

output_df = pd.DataFrame(Outputs).transpose()
output_df.columns = ["output"+str(i) for i in range(1,11)]
output_df

int_out_df_GG_aux = pd.concat([PromptFormat_df.reset_index(drop=True), input_df], axis = 1)
int_out_df_GG = pd.concat([int_out_df_GG_aux.reset_index(drop=True), output_df], axis = 1) 
int_out_df_GG 

,prompt_format,input1,input2,input3,input4,input5,input6,input7,input8,input9,...,output1,output2,output3,output4,output5,output6,output7,output8,output9,output10
0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,290386,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,29-03-86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133819,3,Aquila non capit muscas.,Haec domina toto pendet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,AQUILA NON CAPIT MUSCAS,HAEC DOMINA TOTO PENDET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
133820,3,"Lorem ipsum dolor sit amet, consectetur adipis...",Pellentesque tempor justo a dui varius maximus.,Bona Noctem.,NaN,NaN,NaN,NaN,NaN,NaN,...,LOREM IPSUM DOLOR SIT AMET. CONSECTETUR ADIPIS...,PELLENTESQUE TEMPOR JUSTO A DUI VARIUS MAXIMUS,BONA NOCTEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN
133821,3,Integer malesuada elit et hendrerit fermentum.,Denuntiatores vocate.,Aquila non capit muscas.,NaN,NaN,NaN,NaN,NaN,NaN,...,INTEGER MALESUADA ELIT ET HENDRERIT FERMENTUM,DENUNTIATORES VOCATE,AQUILA NON CAPIT MUSCAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN
133822,3,Quid infantes sumus.,"Praesent sed ipsum euismod, tempor dolor at. v...",Pellentesque tempor justo a dui varius maximus.,NaN,NaN,NaN,NaN,NaN,NaN,...,QUID INFANTES SUMUS,PRAESENT SED IPSUM EUISMOD. TEMPOR DOLOR AT. V...,PELLENTESQUE TEMPOR JUSTO A DUI VARIUS MAXIMUS,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 2.2. GPT-3

In [53]:
df = pd.read_csv("./datasets/GPT3.DataWrangling.csv")
df

,Unnamed: 0,problem,domain,engine,nshot,query,target,output,correct,total_params,prob
0,1,addPunctuation-1,dates,davinci,0,Input: 290386\nOutput:,29-03-86,290104,0,1.750000e+11,0.000888
1,2,addPunctuation-1,dates,davinci,0,Input: 250374\nOutput:,25-03-74,250375,0,1.750000e+11,0.079038
2,3,addPunctuation-1,dates,davinci,0,Input: 170615\nOutput:,17-06-15,170616,0,1.750000e+11,0.232628
3,4,addPunctuation-1,dates,davinci,0,Input: 170905\nOutput:,17-09-05,170905,0,1.750000e+11,0.239137
4,5,addPunctuation-1,dates,davinci,0,Input: 241206\nOutput:,24-12-06,241206,0,1.750000e+11,0.075100
...,...,...,...,...,...,...,...,...,...,...,...
146119,146120,getValue-2,units,ada,10,Input: 7cm\nOutput: 7\n\nInput: 67mm\nOutput: ...,78,78,1,3.500000e+08,0.900076
146120,146121,getValue-2,units,ada,10,Input: 7cm\nOutput: 7\n\nInput: 67mm\nOutput: ...,56676,56776g,0,3.500000e+08,0.821654
146121,146122,getValue-2,units,ada,10,Input: 7cm\nOutput: 7\n\nInput: 67mm\nOutput: ...,56,56,1,3.500000e+08,0.972055
146122,146123,getValue-2,units,ada,10,Input: 7cm\nOutput: 7\n\nInput: 67mm\nOutput: ...,7,7,1,3.500000e+08,0.970601


In [54]:
import re
import numpy as np

N = len(df['query'])
Inputs = [[np.NaN] * N for i in range(11)]
Outputs = [[np.NaN] * N for i in range(10)]

for i in range(N):
    res = [re.split(r"\\n", val) for val in re.split(r'\\n\\n', df['query'][i])]
    for j, lis in enumerate(res):
        if j == df['nshot'][i]:
            Inputs[-1][i] = re.findall(r"Input: (.+)", lis[0])[0]
            
        else:
            Inputs[j][i] = re.findall(r"Input: (.+)", lis[0])[0]
            if len(re.findall(r"Output: (.+)", lis[1])) != 0:
                Outputs[j][i] = re.findall(r"Output: (.+)", lis[1])[0] 

print(Inputs)
print(Outputs)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [55]:
PromptFormat_df = pd.DataFrame([1 for i in range(len(df['query']))])
PromptFormat_df.columns = ["prompt_format"]

input_df = pd.DataFrame(Inputs).transpose()
input_df.columns = ["input"+str(i) for i in range(1,12)]
input_df

output_df = pd.DataFrame(Outputs).transpose()
output_df.columns = ["output"+str(i) for i in range(1,11)]
output_df

int_out_df_GPT3_aux = pd.concat([PromptFormat_df.reset_index(drop=True), input_df], axis = 1)  # concat by columns
int_out_df_GPT3 = pd.concat([int_out_df_GPT3_aux.reset_index(drop=True), output_df], axis = 1)  # concat by columns
int_out_df_GPT3

,prompt_format,input1,input2,input3,input4,input5,input6,input7,input8,input9,...,output1,output2,output3,output4,output5,output6,output7,output8,output9,output10
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146119,1,7cm,67mm,7890.12 km,345 K,21 cm,4235.11 m,90 mol,423 ml,34mm,...,7,67,7890.12,345,21,4235.11,90,423,34,56
146120,1,7cm,67mm,7890.12 km,345 K,21 cm,4235.11 m,90 mol,423 ml,34mm,...,7,67,7890.12,345,21,4235.11,90,423,34,56
146121,1,7cm,67mm,7890.12 km,345 K,21 cm,4235.11 m,90 mol,423 ml,34mm,...,7,67,7890.12,345,21,4235.11,90,423,34,56
146122,1,7cm,67mm,7890.12 km,345 K,21 cm,4235.11 m,90 mol,423 ml,34mm,...,7,67,7890.12,345,21,4235.11,90,423,34,56


# 3. Final integration

In [56]:
df_GPT3 = pd.read_csv("./datasets/GPT3.DataWrangling.csv")
df_Google = pd.read_csv("./datasets/Google.DataWrangling.csv")
df_both_aux = pd.concat([df_GPT3.reset_index(drop=True), df_Google], axis = 0)
df_both = df_both_aux.drop('Unnamed: 0', inplace=False, axis=1)
df_both.reset_index(inplace=True)

In [57]:
int_out_df_both = pd.concat([int_out_df_GPT3.reset_index(drop=True), int_out_df_GG], axis = 0)
int_out_df_both.reset_index(inplace=True)

In [58]:
df_cleaned = pd.concat([df_both.reset_index(drop=True), int_out_df_both], axis = 1)
df_cleaned.drop('index', inplace=True, axis=1)
df_cleaned.rename(columns = {'input11':'test_input'}, inplace = True)
df_cleaned

,problem,domain,engine,nshot,query,target,output,correct,total_params,prob,...,output1,output2,output3,output4,output5,output6,output7,output8,output9,output10
0,addPunctuation-1,dates,davinci,0,Input: 290386\nOutput:,29-03-86,290104,0,1.750000e+11,0.000888,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,addPunctuation-1,dates,davinci,0,Input: 250374\nOutput:,25-03-74,250375,0,1.750000e+11,0.079038,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,addPunctuation-1,dates,davinci,0,Input: 170615\nOutput:,17-06-15,170616,0,1.750000e+11,0.232628,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,addPunctuation-1,dates,davinci,0,Input: 170905\nOutput:,17-09-05,170905,0,1.750000e+11,0.239137,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,addPunctuation-1,dates,davinci,0,Input: 241206\nOutput:,24-12-06,241206,0,1.750000e+11,0.075100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279943,toUpper-2,freetext,BIG-G T=1 8b,2,the input string/s 'Aquila non capit muscas.' ...,'NEMO MORTALIUM OMNIBUS HORIS SAPIT','NEMO MORTIS OMNIBUS HORIS SAPIT.',0,8.721175e+09,0.103250,...,AQUILA NON CAPIT MUSCAS,HAEC DOMINA TOTO PENDET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
279944,toUpper-2,freetext,BIG-G T=1 8b,3,the input string/s 'Lorem ipsum dolor sit amet...,'TIBI BENE VOLO','TIBI BENE VOLO',1,8.721175e+09,0.501232,...,LOREM IPSUM DOLOR SIT AMET. CONSECTETUR ADIPIS...,PELLENTESQUE TEMPOR JUSTO A DUI VARIUS MAXIMUS,BONA NOCTEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN
279945,toUpper-2,freetext,BIG-G T=1 8b,3,the input string/s 'Integer malesuada elit et ...,'FERE LIBENTER HOMINES ID QUOD VOLUNT CREDUNT','FERE LIBENTER HOMINIS ID QUOD VOLUNT CREDIT',0,8.721175e+09,0.001071,...,INTEGER MALESUADA ELIT ET HENDRERIT FERMENTUM,DENUNTIATORES VOCATE,AQUILA NON CAPIT MUSCAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN
279946,toUpper-2,freetext,BIG-G T=1 8b,3,the input string/s 'Quid infantes sumus.' is/a...,'PLACE DICES MIHI LATINE','Place dices mihi LATIN',0,8.721175e+09,0.059498,...,QUID INFANTES SUMUS,PRAESENT SED IPSUM EUISMOD. TEMPOR DOLOR AT. V...,PELLENTESQUE TEMPOR JUSTO A DUI VARIUS MAXIMUS,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
df_cleaned.columns

Index(['problem', 'domain', 'engine', 'nshot', 'query', 'target', 'output',
       'correct', 'total_params', 'prob', 'prompt_format', 'input1', 'input2',
       'input3', 'input4', 'input5', 'input6', 'input7', 'input8', 'input9',
       'input10', 'test_input', 'output1', 'output2', 'output3', 'output4',
       'output5', 'output6', 'output7', 'output8', 'output9', 'output10'],
      dtype='object')

## 3.1. Derivating some syst_features

In [61]:
temperature = []
sparsity = []
model_family = []

for engine in df_cleaned.engine:
    ### Temperature (note: GPT-3 engines used T=0)
    temperature.append(1) if "T=1" in engine else temperature.append(0)
    ### sparsity
    sparsity.append(1) if "sparse" in engine else sparsity.append(0)
    ### model_family
    if "sparse" in engine:
        model_family.append("BIG-G sparse")
    elif "T=0" in engine:
        model_family.append("BIG-G T=0")
    elif "T=1" in engine:
        model_family.append("BIG-G T=1")
    else:
        model_family.append("GPT-3") 

In [62]:
import collections
print(collections.Counter(temperature))
print(collections.Counter(sparsity))
print(collections.Counter(model_family))

Counter({0: 232716, 1: 47232})
Counter({0: 240588, 1: 39360})
Counter({'GPT-3': 146124, 'BIG-G T=0': 47232, 'BIG-G T=1': 47232, 'BIG-G sparse': 39360})


In [63]:
df_cleaned.insert(10, column="temperature", value=temperature)
df_cleaned.insert(11, column="sparsity", value=sparsity)
df_cleaned.insert(12, column="model_family", value=model_family)

In [64]:
df_cleaned.columns

Index(['problem', 'domain', 'engine', 'nshot', 'query', 'target', 'output',
       'correct', 'total_params', 'prob', 'temperature', 'sparsity',
       'model_family', 'prompt_format', 'input1', 'input2', 'input3', 'input4',
       'input5', 'input6', 'input7', 'input8', 'input9', 'input10',
       'test_input', 'output1', 'output2', 'output3', 'output4', 'output5',
       'output6', 'output7', 'output8', 'output9', 'output10'],
      dtype='object')

In [65]:
df_cleaned.to_csv('./datasets/GPT3.and.Google.DataWrangling.csv')  